In [3]:
import csv
import os
from datetime import datetime, date
import matplotlib.pyplot as plt
from collections import defaultdict
DATA_FILE = 'expenses.csv'
DATE_FMT = '%d-%m-%Y'

In [4]:
def parse_date(s: str) -> date:
    s = s.strip()
    candidates = [
        "%d-%m-%Y %H:%M:%S",
        "%d-%m-%Y",
        "%Y-%m-%d %H:%M:%S",
        "%Y-%m-%d",
        "%d/%m/%Y",
    ]
    for fmt in candidates:
        try:
            return datetime.strptime(s, fmt).date()
        except ValueError:
            continue
    raise ValueError(
        f"Date '{s}' does not match supported formats "
        "(dd-mm-yyyy, yyyy-mm-dd, with/without time)."
    )

def format_date(d: date) -> str:
    return d.strftime(DATE_FMT)

In [5]:
def initialize_file():
    # Ensure the CSV file exists with the correct header.
    if not os.path.exists(DATA_FILE):
        with open(DATA_FILE, 'w', newline='') as f:
            writer = csv.writer(f)
            writer.writerow(['date', 'category', 'amount', 'description'])


In [6]:
def load_expenses():
    expenses = []
    if not os.path.exists(DATA_FILE):
        return expenses

    with open(DATA_FILE, 'r', newline='') as f:
        reader = csv.DictReader(f)
        # Expect headers: date, category, amount, description
        for row in reader:
            if not row.get('date'):
                continue
            try:
                d = parse_date(row['date'])
            except ValueError as e:
                # If a bad row sneaks in, skip with a message
                print(f"Skipping row with bad date '{row.get('date')}': {e}")
                continue
            try:
                amt = float(row['amount'])
            except (ValueError, TypeError):
                print(f"Skipping row with bad amount '{row.get('amount')}'.")
                continue
            expenses.append({
                'date': d,
                'category': row.get('category', '').strip(),
                'amount': amt,
                'description': row.get('description', '').strip()
            })
    return expenses


In [7]:
def save_expenses(expenses):
    with open(DATA_FILE, 'w', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(['date', 'category', 'amount', 'description'])
        for exp in expenses:
            writer.writerow([
                format_date(exp['date']),  # dd-mm-yyyy
                exp['category'],
                exp['amount'],
                exp['description']
            ])

# ---------- CRUD ----------
def add_expense():
    date_str = input('Enter date (dd-mm-yyyy) [leave blank for today]: ').strip()
    if date_str:
        d = parse_date(date_str)
    else:
        d = date.today()

    category = input('Enter category: ').strip()
    amount = float(input('Enter amount: ').strip())
    description = input('Enter description: ').strip()

    expenses = load_expenses()
    expenses.append({
        'date': d,
        'category': category,
        'amount': amount,
        'description': description
    })
    save_expenses(expenses)
    print('Expense added!')

def view_expenses():
    expenses = load_expenses()
    if not expenses:
        print('No expenses found.')
        return
    for i, exp in enumerate(expenses, start=1):
        print(f"{i}. {format_date(exp['date'])} | {exp['category']} | ₹{exp['amount']} | {exp['description']}")

def delete_expense():
    expenses = load_expenses()
    if not expenses:
        print('No expenses to delete.')
        return
    view_expenses()
    try:
        idx = int(input('Enter record number to delete: ').strip()) - 1
    except ValueError:
        print('Invalid input.')
        return
    if 0 <= idx < len(expenses):
        removed = expenses.pop(idx)
        save_expenses(expenses)
        print(f" Deleted: {format_date(removed['date'])} | {removed['category']} | ₹{removed['amount']} | {removed['description']}")
    else:
        print('Invalid record number.')

def update_expense():
    expenses = load_expenses()
    if not expenses:
        print('No expenses to update.')
        return
    view_expenses()
    try:
        idx = int(input('Enter record number to update: ').strip()) - 1
    except ValueError:
        print('Invalid input.')
        return
    if 0 <= idx < len(expenses):
        exp = expenses[idx]
        print('Press Enter to keep current value.')
        date_str = input(f"Date ({format_date(exp['date'])}): ").strip()
        if date_str:
            exp['date'] = parse_date(date_str)  # accepts dd-mm-yyyy or yyyy-mm-dd

        cat = input(f"Category ({exp['category']}): ").strip()
        if cat:
            exp['category'] = cat

        amt = input(f"Amount ({exp['amount']}): ").strip()
        if amt:
            exp['amount'] = float(amt)

        desc = input(f"Description ({exp['description']}): ").strip()
        if desc:
            exp['description'] = desc

        save_expenses(expenses)
        print('Expense updated!')
    else:
        print('Invalid record number.')

In [8]:
def plot_summary():
    """
    Generate two plots:
    1) Line chart of monthly totals.
    2) Pie chart of expenses by category.
    """
    expenses = load_expenses()
    if not expenses:
        print('No data to plot.')
        return

    # Monthly totals (key = first day of month as date object)
    monthly = defaultdict(float)
    for exp in expenses:
        month_key = date(exp['date'].year, exp['date'].month, 1)
        monthly[month_key] += exp['amount']
    months_sorted = sorted(monthly.keys())
    x_labels = [m.strftime('%b %Y') for m in months_sorted]
    totals = [monthly[m] for m in months_sorted]

    plt.figure()
    plt.plot(x_labels, totals, marker='o')
    plt.title('Monthly Expenses')
    plt.xlabel('Month')
    plt.ylabel('Amount (₹)')
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

    # Category breakdown
    cat_totals = defaultdict(float)
    for exp in expenses:
        cat_totals[exp['category']] += exp['amount']

    plt.figure()
    plt.pie(
        list(cat_totals.values()),
        labels=list(cat_totals.keys()),
        autopct='%1.1f%%'
    )
    plt.title('Expenses by Category')
    plt.show()

def monthly_category_summary():
    expenses = load_expenses()
    if not expenses:
        print('No data found.')
        return

    summary = defaultdict(lambda: defaultdict(float))  # summary[month_name][category] = total
    for exp in expenses:
        month_name = exp['date'].strftime("%B %Y")
        summary[month_name][exp['category']] += exp['amount']

    for month in sorted(summary.keys(), key=lambda s: datetime.strptime(s, "%B %Y")):
        print(f"\n {month}")
        for category, total in sorted(summary[month].items()):
            print(f"   {category}: ₹{total:.2f}")

In [9]:
def main():
    initialize_file()
    while True:
        print('''
=== Personal Expense Tracker ===
1) Add Expense
2) View Expenses
3) Delete Expense
4) Update Expense
5) Plot Summary
6) Total of Month (Category-wise)
7) Exit
        ''')
        choice = input('Choose an option: ').strip()
        if choice == '1':
            add_expense()
        elif choice == '2':
            view_expenses()
        elif choice == '3':
            delete_expense()
        elif choice == '4':
            update_expense()
        elif choice == '5':
            plot_summary()
        elif choice == '6':
            monthly_category_summary()
        elif choice == '7':
            print('Goodbye!')
            break
        else:
            print('Invalid choice, try again.')

if __name__ == '__main__':
    main()


=== Personal Expense Tracker ===
1) Add Expense
2) View Expenses
3) Delete Expense
4) Update Expense
5) Plot Summary
6) Total of Month (Category-wise)
7) Exit
        
Choose an option: 1
Enter date (dd-mm-yyyy) [leave blank for today]: 
Enter category: travel
Enter amount: 5000
Enter description: delhi
Expense added!

=== Personal Expense Tracker ===
1) Add Expense
2) View Expenses
3) Delete Expense
4) Update Expense
5) Plot Summary
6) Total of Month (Category-wise)
7) Exit
        
Choose an option: 2
1. 09-04-2025 | food | ₹2000.0 | ich
2. 10-06-2025 | shoping | ₹20000.0 | kpde liye
3. 30-08-2026 | food | ₹200.0 | dosa
4. 29-08-2026 | food | ₹300.0 | rice
5. 30-08-2025 | travel | ₹5000.0 | delhi

=== Personal Expense Tracker ===
1) Add Expense
2) View Expenses
3) Delete Expense
4) Update Expense
5) Plot Summary
6) Total of Month (Category-wise)
7) Exit
        
Choose an option: 6

 April 2025
   food: ₹2000.00

 June 2025
   shoping: ₹20000.00

 August 2025
   travel: ₹5000.00

 A